In [ ]:
#Librairies nécessaires
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:.2f}')

In [ ]:
#############################################################
#############################################################
df = pd.read_excel('data/input_fichier.xlsx') #Il faudra changer le chemin et/ou le nom du fichier ici d'un mois/trimestre/année à l'autre
#############################################################
#############################################################

#Ajout d'une colonne ID_global au cas où Index n'est pas unique (s'il y a plusieurs mois dans la requête par exemple)
df['ID_global'] = df.index

#On filtre les lignes où 'Fcontrole' vaut 'N'
df_filtered = df[df['Fcontrole'] == 'N'].copy()

#On enregistre les lignes où 'Fcontrole' vaut 'O' pour plus tard
df_flag_O = df[df['Fcontrole'] == 'O'].copy()

#On définit une fonction pour attribuer une strate à chaque ligne
def assign_strate(row):
    #On définit les conditions pour l'automatisation
    if row['Nagent'].startswith('I'):
        if row['Nprestation'] in ['Liste des critères']:
            return 'Auto1'
        elif row['Nprestation'] in ['Liste des critères']:
            return 'Auto2'
    
    #On définit les conditions pour l'acte de ville
    if row['Nagent'][0] not in ['A', 'I'] and row['LTprestation'] == 'Acte de ville':
        if row['Crégion'] in [1, 2, 3]:
            if row['Nprestation'] in ['Liste des critères']:
                return 'AV1'
            elif row['Nprestation'] in ['Liste des critères']:
                return 'AV2'
        elif row['Crégion'] == 0:
            if row['Nprestation'] in ['Liste des critères']:
                return 'AV3'
            elif row['Nprestation'] in ['Liste des critères']:
                return 'AV4'
        elif row['Crégion'] in [4, 5]:
            return 'AV5'

    #On définit les conditions pour l'hospitalisation
    if row['Nagent'][0] not in ['A', 'I'] and row['LTprestation'] == 'Hospitalisation':
        if row['Crégion'] == 0:
            return 'H1'
        else:
            return 'H2'

    #Si aucune condition ne correspond
    return None

#On applique la fonction sur les lignes filtrées
df_filtered['Strate'] = df_filtered.apply(assign_strate, axis=1)
print('Strates assignées')

#On définit une nouvelle fonction pour attribuer une 'Strate décompte' dans les cas où un décompte serait représenté dans plusieurs strates
#On attribue la strate la plus représentée en terme de Mremboursé
def assign_strate_decompte(decompte):
    #On filtre les lignes où la strate est définie
    decompte_valid = decompte[decompte['Strate'].notna()]

    #Si le décompte n'est pas vide après filtrage, on peut calculer la somme des montants remboursés par strate
    if not decompte_valid.empty:
        sum_by_strate = decompte_valid.groupby('Strate')['Mremboursé'].sum()
        #On trouve la strate avec la somme la plus élevée
        max_strate = sum_by_strate.idxmax()
        #On attribue cette strate à toutes les lignes du décompte
        decompte['Strate décompte'] = max_strate
    else:
        #Si le groupe est vide ou n'a pas de strate valide
        decompte['Strate décompte'] = None
    
    return decompte

#On applique la fonction sur chaque décompte
df_final = df_filtered.groupby('Ndecompte').apply(assign_strate_decompte)
df_final = df_final.reset_index(drop=True)
print('Strates décompte assignées')

In [ ]:
#On réinjecte les lignes où 'Fcontrole' vaut 'O'
df_concat = pd.concat([df_final, df_flag_O], ignore_index=True)
df_concat = df_concat.sort_values('ID_global')
df_concat

In [ ]:
#Afin d'éviter d'exporter tout le tableau (processus assez long), on n'exporte que les 2 dernières colonnes qu'on pourra copier/coller pour ajouter à la requête (+ ou - 6 secondes par mois)

strates = df_concat.iloc[:, -2:]
#############################################################
#############################################################
strates.to_excel('data/output_fichier.xlsx', index=False) #Il faudra changer le chemin d'enregistrement d'un trimestre/année à l'autre
#############################################################
#############################################################

In [ ]:
#On compte le nombre de décomptes à contrôler a posteriori

#On filtre pour ne conserver que les montants positifs
df_concat_filtered = df_concat[df_concat['Mremboursé'] > 0]

#On créé la colonne 'Processus' en fonction de 'Strate décompte'
conditions = [
    df_concat_filtered['Strate décompte'].isin(['Auto1', 'Auto2']), #Automatisation
    df_concat_filtered['Strate décompte'].isin(['AV1', 'AV2', 'AV3', 'AV4', 'AV5']), #Acte de ville
    df_concat_filtered['Strate décompte'].isin(['H1', 'H2']) #Hospitalisation
]

choices = ['Automatisation', 'Acte de ville', 'Hospitalisation']

df_concat_filtered = df_concat_filtered.copy()
df_concat_filtered['Processus'] = np.select(conditions, choices, default='Inconnu')

#Nouvelle colonne pour extraire le mois
df_concat_filtered['Mois'] = df_concat_filtered['Date de règlement'].dt.to_period('M')

#On groupe par 'Processus', 'Strate décompte' et 'Mois', pour calculer le nombre de décomptes uniques
df_resultat = df_concat_filtered.groupby(['Processus', 'Strate décompte', 'Mois'])['Ndecompte'].nunique().reset_index()

#On calcule la somme de décomptes pour chaque 'Processus' par 'Mois'
somme_par_processus_mois = df_resultat.groupby(['Processus', 'Mois'])['Ndecompte'].transform('sum')

#Création de la colonne 'Echantillon' qui servira pour la suite
df_resultat['Echantillon'] = ((df_resultat['Ndecompte'] / somme_par_processus_mois) * 50).round(0).astype(int)
df_resultat

In [ ]:
#Dictionnaires pour stocker les dataframes par processus et par mois
controle_post_auto = {}
controle_post_adv = {}
controle_post_hospi = {}

df_final['Mois'] = df_final['Date de règlement'].dt.to_period('M')
df_flag_O['Mois'] = df_flag_O['Date de règlement'].dt.to_period('M')

df_final = df_final.merge(df_concat_filtered[['ID_global', 'Processus']], on='ID_global', how='left')

#Boucle sur les mois uniques présents dans df_resultat
for mois in df_resultat['Mois'].unique():
    #On filtre les données du mois en cours
    df_final_filtered_mois = df_final[(df_final['Mremboursé'] > 0) & (df_final['Mois'] == mois)]
    
    #Boucle sur les processus
    for processus in ['Automatisation', 'Acte de ville', 'Hospitalisation']:
        df_processus = df_final_filtered_mois[df_final_filtered_mois['Processus'] == processus]
        
        #On génère les quotas pour le mois et le processus en cours
        quotas_final = df_resultat[(df_resultat['Mois'] == mois) & (df_resultat['Processus'] == processus)] \
            .set_index('Strate décompte')['Echantillon'].to_dict()
        
        #Ajout de 30 décomptes supplémentaires à contrôler en hospi ETR (strate H2)
        if 'H2' in quotas_final:
            quotas_final['H2'] += 30

        #On sélectionne aléatoirement des décomptes selon les quotas
        selected_decomptes_final = []
        for strate, n in quotas_final.items():
            decomptes = df_processus[df_processus['Strate décompte'] == strate]['Ndecompte'].unique()
            if len(decomptes) > n:
                selected = pd.Series(decomptes).sample(n, random_state=42).tolist()
            else:
                selected = decomptes.tolist()
            selected_decomptes_final.extend(selected)

        #On extrait les lignes correspondantes et on les stocke dans le bon dictionnaire
        df_selected = df_processus[df_processus['Ndecompte'].isin(selected_decomptes_final)]
        
        if processus == 'Automatisation':
            controle_post_auto[f"{mois}_contrôle_post_auto"] = df_selected
        elif processus == 'Acte de ville':
            controle_post_adv[f"{mois}_contrôle_post_adv"] = df_selected
        elif processus == 'Hospitalisation':
            controle_post_hospi[f"{mois}_contrôle_post_hospi"] = df_selected

#Affichage des résultats
for key, df in {**controle_post_auto, **controle_post_adv, **controle_post_hospi}.items():
    print(f"{key}, nombre de décomptes : {df['Ndecompte'].nunique()}")
    print('Nombre de lignes :', df.shape[0])
    display(df.head(5))

In [ ]:
controle_complementaire_par_mois = {}

#Boucle sur chaque mois unique
for mois in df_flag_O['Mois'].unique():
    #On filtre les données du mois en cours
    df_filtered_mois = df_flag_O[(df_flag_O['Mremboursé'] > 0) & (df_flag_O['Mois'] == mois)]

    #On initialise la liste des décomptes sélectionnés
    selected_decomptes_flag_O = []

    #Automatisation (Nagent commence par 'I')
    decomptes_auto = df_filtered_mois[df_filtered_mois['Nagent'].str.startswith('I')]['Ndecompte'].unique()
    selected_decomptes_flag_O.extend(pd.Series(decomptes_auto).sample(min(len(decomptes_auto), 3), random_state=42).tolist())

    #Acte de ville
    decomptes_adv = df_filtered_mois[
        (df_filtered_mois['LTprestation'] == 'Acte de ville') & 
        (~df_filtered_mois['Nagent'].str.startswith('I'))
    ]['Ndecompte'].unique()
    selected_decomptes_flag_O.extend(pd.Series(decomptes_adv).sample(min(len(decomptes_adv), 3), random_state=42).tolist())

    #Hospitalisation
    decomptes_hospi = df_filtered_mois[df_filtered_mois['LTprestation'] == 'Hospitalisation']['Ndecompte'].unique()
    selected_decomptes_flag_O.extend(pd.Series(decomptes_hospi).sample(min(len(decomptes_hospi), 3), random_state=42).tolist())

    #On extrait les lignes correspondantes et on les stocke dans un dataframe
    controle_complementaire_par_mois[f"{mois}_contrôle_complémentaire"] = df_filtered_mois[df_filtered_mois['Ndecompte'].isin(selected_decomptes_flag_O)]

#Affichage des résultats
for mois, df_controle in controle_complementaire_par_mois.items():
    print(f"{mois}, nombre de décomptes : {df_controle['Ndecompte'].nunique()}")
    print('Nombre de lignes :', df_controle.shape[0])
    display(df_controle)

In [ ]:
controle_fronting_par_mois = {}

#Filtrage des décomptes fronting
df_fronting = df_final[
    (df_final['Nagent'].str.startswith('A')) &
    (df_final['SF/E'] == 'SOINS ETRANGER')
]

#Boucle sur chaque mois unique
for mois in df_fronting['Mois'].unique():
    df_mois = df_fronting[df_fronting['Mois'] == mois]  # Filtrer par mois

    #Boucle sur chaque partenaire
    for partenaire in df_mois['Ndestinataire'].unique():
        df_partenaire = df_mois[df_mois['Ndestinataire'] == partenaire]

        #On agrège le montant total remboursé par décompte
        df_agg = df_partenaire.groupby(['Ndecompte', 'LTprestation'])['Mremboursé'].sum().reset_index()

        #On compte le nombre de décomptes uniques par processus
        nb_hospi = len(df_agg[df_agg['LTprestation'] == 'Hospitalisation'])
        nb_adv = len(df_agg[df_agg['LTprestation'] == 'Acte de ville'])

        #Si un seul type est disponible, prendre les 5 dans ce type
        if nb_hospi == 0:
            quota_hospi, quota_adv = 0, 5
        elif nb_adv == 0:
            quota_hospi, quota_adv = 5, 0
        else:
            #Sinon, calcul des quotas proportionnels tout en respectant 1 minimum et 4 maximum par type
            ratio_hospi = nb_hospi / (nb_hospi + nb_adv)
            quota_hospi = max(1, min(4, round(ratio_hospi * 5)))
            quota_adv = 5 - quota_hospi

        #Sélection des décomptes avec les montants remboursés les plus élevés
        top_hospi = df_agg[df_agg['LTprestation'] == 'Hospitalisation'] \
            .sort_values(by='Mremboursé', ascending=False) \
            .head(quota_hospi)['Ndecompte']

        top_adv = df_agg[df_agg['LTprestation'] == 'Acte de ville'] \
            .sort_values(by='Mremboursé', ascending=False) \
            .head(quota_adv)['Ndecompte']

        #On rejoint les lignes complètes avec la sélection des Ndecomptes
        df_selected = df_partenaire[df_partenaire['Ndecompte'].isin(pd.concat([top_hospi, top_adv]))]

        #Stockage des résultats
        key = f"{mois}_contrôle_fronting"
        if key in controle_fronting_par_mois:
            controle_fronting_par_mois[key] = pd.concat([controle_fronting_par_mois[key], df_selected])
        else:
            controle_fronting_par_mois[key] = df_selected

#Affichage des résultats
for mois, df_controle in controle_fronting_par_mois.items():
    print(f"{mois}, nombre de décomptes : {df_controle['Ndecompte'].nunique()}")
    print('Nombre de lignes :', df_controle.shape[0])
    display(df_controle.head(10))

In [ ]:
import os

#Définition du dossier de sortie
output_folder = "../Contrôle a posteriori"
os.makedirs(output_folder, exist_ok=True)

#Dictionnaires contenant les dataframes à exporter
controle_dicts = {"auto": controle_post_auto,
                  "adv": controle_post_adv,
                  "hospi": controle_post_hospi}

#On exporte les fichiers contrôle a posteriori (sans les 21 dernières colonnes)
for processus, dict_mois in controle_dicts.items():
    for mois_complet, df_post in dict_mois.items():
        if df_post is not None and not df_post.empty:
            mois = mois_complet.split("_")[0]  #On extrait uniquement l'année et le mois
            mois_folder = os.path.join(output_folder, mois)  
            os.makedirs(mois_folder, exist_ok=True)

            file_path_post = os.path.join(mois_folder, f"{mois_complet}.xlsx")
            df_post.iloc[:, :-21].to_excel(file_path_post, index=False)
            print(f"Fichier exporté : {file_path_post}")

#On exporte les fichiers contrôle complémentaire (sans les 18 dernières colonnes)
for mois_complet, df_complementaire in controle_complementaire_par_mois.items():
    if df_complementaire is not None and not df_complementaire.empty:
        mois = mois_complet.split("_")[0]  #On extrait uniquement l'année et le mois
        mois_folder = os.path.join(output_folder, mois)  
        os.makedirs(mois_folder, exist_ok=True)

        file_path_complementaire = os.path.join(mois_folder, f"{mois_complet}.xlsx")
        df_complementaire.iloc[:, :-18].to_excel(file_path_complementaire, index=False)
        print(f"Fichier exporté : {file_path_complementaire}")

#On exporte les fichiers fronting (sans les 21 dernières colonnes)
for mois_complet, df_fronting in controle_fronting_par_mois.items():
    if df_fronting is not None and not df_fronting.empty:
        mois = mois_complet.split("_")[0]  #On extrait uniquement l'année et le mois
        mois_folder = os.path.join(output_folder, mois)  
        os.makedirs(mois_folder, exist_ok=True)

        file_path_fronting = os.path.join(mois_folder, f"{mois_complet}.xlsx")
        df_fronting.iloc[:, :-21].to_excel(file_path_fronting, index=False)
        print(f"Fichier exporté : {file_path_fronting}")